# Temporal Analyzers
Notebook used to look at anything that requires parsing through a high range of snapshots, i.e. a large amount of time.<br>

## 1. Load Utilities

In [ ]:
# Loading libraries and key coordinates
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import stats

##### import functions from utilities.py #####
from scripts.notebooks.utilities import Temp_S

################################################
##### PHYSICAL CONSTANTS #####
gamma = 5/3
kb = 1.3807e-16 # Boltzmann Constant in CGS or erg/K
seconds_in_myrs = 3.15576e+13
seconds_in_yrs = 3.154e+7
sb_constant_cgs = 5.670374419e-5 # ergs/cm^2/K^4/s
from scripts.notebooks.utilities import PROTON_MASS_GRAMS # 1.67262192e-24 
from scripts.notebooks.utilities import GRAVITIONAL_CONSTANT_IN_CGS #  6.6738e-8
from scripts.notebooks.utilities import HUBBLE #  3.2407789e-18

##### UNITS #####
UnitVelocity_in_cm_per_s = 1e5 # 10 km/sec 
UnitLength_in_cm = 3.085678e21 # 1 kpc
UnitMass_in_g  = 1.989e33 # 1 solar mass
UnitTime_in_s = UnitLength_in_cm / UnitVelocity_in_cm_per_s # 3.08568e+16 seconds 
UnitEnergy_in_cgs = UnitMass_in_g * pow(UnitLength_in_cm, 2) / pow(UnitTime_in_s, 2) # 1.9889999999999999e+43 erg
UnitDensity_in_cgs = UnitMass_in_g / pow(UnitLength_in_cm, 3) # 6.76989801444063e-32 g/cm^3
UnitPressure_in_cgs = UnitMass_in_g / UnitLength_in_cm / pow(UnitTime_in_s, 2) # 6.769911178294542e-22 barye
UnitNumberDensity = UnitDensity_in_cgs/PROTON_MASS_GRAMS
UnitEnergyDensity = UnitEnergy_in_cgs/pow(UnitLength_in_cm, 3)

G_code = GRAVITIONAL_CONSTANT_IN_CGS/(pow(UnitLength_in_cm,3) * pow(UnitMass_in_g, -1) * pow(UnitTime_in_s, -2)) 
Hubble_code = HUBBLE * UnitTime_in_s # All.Hubble in Arepo

###### KEY SIMULATION PARAMETERS ######
boxsize = 100
midpoint = boxsize/2 
center_boxsize = 10
center_boxsize_large = 15 
cells_per_dim = 301 
cells_per_dim_large = 451 # for the MW plots

# Ex: 80 85 90 95 100
first = 0
last = 100
step = 5
dx = center_boxsize/300
eps = dx/1e6
angle_l = 60
z_cut = 0.5
T_COLD_MAX = 3e4
n_bins = 300

plt.style.use("seaborn-paper")
color_map = plt.get_cmap('turbo')
##### FOLDERS TO ANALYZE ######
outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_long_bursts/", "./outflow_cooling/output_short_bursts/", "./outflow_cooling/output_short_fast_bursts/"]
labeling = [r"Continuous", r"$\rm t_{burst} = 5 \, Myr,\,\, t_{rest} =  10 \, Myr$", 
          r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  5 \, Myr$", r"$\rm t_{burst} = 1 \, Myr,\,\, t_{rest} =  1 \, Myr$"]

# outputs = ["./outflow_cooling/output_PIE_fid/", "./outflow_cooling/output_high_alpha/", "./outflow_cooling/output_high_beta/", "./outflow_cooling/output_high_alpha_high_beta/"]
# labeling = []

##############################

log10temp_bins = np.linspace(3, 8, 3000)
vel_bins = np.linspace(-50, 2000, 3000)
log10nd_bins = np.linspace(-5, 3, 3000)

coloring = color_map(np.linspace(0, 1, len(outputs)))


## 2. Data Collection
Loops over every snapshot and collects gas quantities over time. 

In [ ]:

time_values = []
fraction_cold_gas_time = [] # The fraction of cold gas mass over all snapshots.

distances_bins = np.linspace(z_cut, 50, 51) # 1 kpc for each bin spanning from 0 to 50 kpc 
fraction_cold_gas_r = np.zeros(shape=( len(outputs), len(distances_bins) - 1) )  #  The mean profule for r 

r_bins = np.linspace(0, 20, 601) 
radial_velocity_over_time = np.zeros(shape=( len(outputs), len(r_bins) - 1) ) 
density_over_time = np.zeros(shape=( len(outputs), len(r_bins) - 1) )
temperature_over_time = np.zeros(shape=( len(outputs), len(r_bins) - 1) )

coloring = color_map(np.linspace(0, 1, len(outputs)))

for o, output in enumerate(outputs):
    print(output)
    data = {}
    fc_time = []

    rad_v_time = []
    nd_time = []
    temp_time = []

    times_def = []

    for i in np.arange(first, last+step, step):
        filename = output + "snap_%03d.hdf5" % i
        with h5py.File(filename,'r') as f:
            for key in f['PartType0']:
                data[key] = f['PartType0'][key][()]
            header = dict(f['Header'].attrs)
            parameters = dict(f['Parameters'].attrs)

        masses = data["Masses"]
        t = header["Time"] # in units of 1 gyr 
        R = parameters["injection_radius"]
        alpha = parameters["E_load"]
        beta = parameters["M_load"]
        # t_yr = t*UnitTime_in_s/seconds_in_yrs # Code Units * Seconds/Code Units * years/seconds
        xc_io = data["Coordinates"][:,0]
        yc_io = data["Coordinates"][:,1]
        zc_io = data["Coordinates"][:,2]
        vel_x = data["Velocities"][:,0]
        vel_y = data["Velocities"][:,1]
        vel_z = data["Velocities"][:,2]
        density = data["Density"]
        number_density = density*UnitNumberDensity
        internal_energy = data["InternalEnergy"]
        abundance = data["ElectronAbundance"]
        temperature = Temp_S(abundance, internal_energy)

        rad_x = xc_io - 0.5*boxsize
        rad_y = yc_io - 0.5*boxsize
        rad_z = zc_io - 0.5*boxsize
        radius = np.sqrt(rad_x**2+rad_y**2+rad_z**2)

        radial_velocity = (vel_x*rad_x + vel_y*rad_y + vel_z*rad_z)/(radius + eps)


        theta = np.arccos(np.abs(rad_z)/(radius + eps))*180/np.pi 
        angular_region = (np.abs(theta) <= angle_l) & (np.abs(rad_z) >= z_cut) # Excludes anything with absolute angles greater than 60 and the disk.
        angular_region_cold = (angular_region) & (temperature <= T_COLD_MAX)# 1e3.4 K

        if i == first: 
            # nd_l = np.percentile(number_density[rad_z >= 0.5], 3)
            nd_h = np.percentile(number_density[rad_z >= 0.5], 99.85)
            print(nd_h)
            # t_l = np.percentile(temperature[rad_z >= 0.5], 3)
            t_l = np.percentile(temperature[rad_z >= 0.5], 0.15)
            print(t_l)
        
        # Background cells are low density, hot, and slow
        # remove any cells that 3 std higher than the mean number density, 3 std lower than the mean temperature, and low velocity
        bg_cells = (number_density <= nd_h) & (temperature >= t_l) & (np.abs(radial_velocity) <= 40) # Gets rids of most BG cells.
        
        angular_bg_mass = (angular_region) & (~bg_cells)
        angular_bg_mass_cold = angular_region_cold & (~bg_cells)
        total_fract_cold = np.sum(masses[angular_bg_mass_cold])/np.sum(masses[angular_bg_mass])

        cold_gas_binned , dbins = np.histogram(radius[angular_bg_mass_cold], bins=distances_bins, weights=masses[angular_bg_mass_cold])
        gas_frac_snapshot = cold_gas_binned/np.sum(masses[angular_bg_mass])

        fc_time.append(total_fract_cold)
        times_def.append(t*1000) # t in megayears

        total_masses_bicone = np.sum(masses[angular_bg_mass])
        # fracts_cold = masses[(cold) & (angular_region)]/(masses[angular_region])

        fraction_cold_gas_r[o] += gas_frac_snapshot  

        rad_v, v_edge, _ = stats.binned_statistic(radius[angular_region], radial_velocity[angular_region], statistic="median", bins=r_bins, range=(0, 20))
        nd_rad, n_edge, _ = stats.binned_statistic(radius[angular_region], number_density[angular_region], statistic="median", bins=r_bins, range=(0, 20))
        Temp_rad, T_edge, _ = stats.binned_statistic(radius[angular_region], temperature[angular_region], statistic="median",  bins=r_bins, range=(0, 20))

        radial_velocity_over_time[o] += rad_v
        density_over_time[o] += nd_rad
        temperature_over_time[o] += Temp_rad

    # labeling.append(r"$\alpha = %.1f, \beta = %.1f$" % (parameters["E_load"], parameters["M_load"]))

    time_values.append(times_def)
    fraction_cold_gas_time.append(fc_time)
    
radial_velocity_over_time /= (last - first)/step + 1 # (100 - 90)/5 = 2, so we need to add a +1 
density_over_time /= (last - first)/step + 1
temperature_over_time /= (last - first)/step + 1

fraction_cold_gas_r /= (last - first)/step + 1


## 3. Plots
### Mean radial profiles, averaged over all time

In [ ]:
from scripts.notebooks.utilities import mean_molecular_weight
from scipy import optimize
from scripts.notebooks.utilities import sol_in
from scripts.notebooks.utilities import sol_out
M_load = parameters["M_load"]
E_load = parameters["E_load"]
R = parameters["injection_radius"]
sfr = parameters["sfr"]
r_an = np.linspace(0.001, 20, 1000)

r_in = r_an[np.where(r_an <= R)]
r_out = r_an[np.where(r_an > R)]

s_in_yr = 3.154e+7
grams_in_M_sun = 1.989e33
M_dot_wind = sfr*M_load # solar masses per 1 year -> get this in grams per second 
M_dot_cm = (M_dot_wind*UnitMass_in_g)/s_in_yr # grams/second
E_dot_wind = E_load*3e41*sfr # this is in ergs/second 

M_dot_code = M_dot_wind/(UnitMass_in_g/grams_in_M_sun)*(UnitTime_in_s/s_in_yr)
E_dot_code = E_dot_wind/UnitEnergy_in_cgs*UnitTime_in_s

M1 = optimize.fsolve(sol_in, x0=np.full(len(r_in), 0.001), args=(r_in, R))
M2 = optimize.fsolve(sol_out, x0=np.full(len(r_out), 100), args=(r_out, R))
M = np.concatenate([M1, M2])

v_an = (M*np.sqrt(E_dot_code/M_dot_code)*(((gamma - 1)*M**2 + 2)/(2*(gamma - 1)))**(-0.5)) # this is in code units
v_in = v_an[np.where(r_an <= R)]
v_out = v_an[np.where(r_an > R)]
cs = np.sqrt( (E_dot_code/M_dot_code)*(((gamma - 1)*M**2 + 2)/(2*(gamma - 1)))**(-1))
cs_cm = cs*(UnitVelocity_in_cm_per_s) 

rho_in = M_dot_code/(4*np.pi*v_in)*(r_in/R**3)*UnitDensity_in_cgs
rho_out = M_dot_code/(4*np.pi*v_out)*1/r_out**2*UnitDensity_in_cgs


rho_an = np.concatenate([rho_in, rho_out])
rho_n = np.concatenate([rho_in, rho_out])/PROTON_MASS_GRAMS # rho/(proton mass)

pressure_an = ((rho_an*cs_cm**2)/gamma)/kb # -> (g/cm^3* cm^2/s^2) -> p/kb 

# P/kb = rho_an/(mean molecular weight * proton mass) * T = P/kb = rho/(proton mass) * 1/mean molecular weight * T
## T = pressure_an/(rho_n)* mean molecular weight
temp_an = pressure_an/(rho_n)*(mean_molecular_weight(1)/PROTON_MASS_GRAMS) # keep the mean molecular mass the same. 

labeling = [r"Continuous", r"$\rm t_{b} = 5 \, Myr,\, t_{r} =  10 \, Myr$", 
          r"$\rm t_{b} = 1 \, Myr,\, t_{r} =  5 \, Myr$", r"$\rm t_{b} = 1 \, Myr,\, t_{r} =  1 \, Myr$"]

print((last - first)/step)
fig = plt.figure(figsize=(8, 10))
fig.set_rasterized(True)
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)

for o, output in enumerate(outputs):
    v_centers = 0.5 * (v_edge[:-1] + v_edge[1:])
    nd_centers =  0.5 * (n_edge[:-1] + n_edge[1:])
    T_centers =  0.5 * (T_edge[:-1] + T_edge[1:])

    ax1.plot(v_centers, radial_velocity_over_time[o], label=labeling[o], linestyle="solid", color=coloring[o])
    ax2.semilogy(nd_centers, density_over_time[o], label=labeling[o], linestyle="solid", color=coloring[o])
    ax3.semilogy(T_centers, temperature_over_time[o], label=labeling[o], linestyle="solid", color=coloring[o])

ax3.semilogy(r_an,temp_an, color="blue", linestyle="dashed", label="CC85")
ax1.text(1, 300, "Time Range = %0.0f - %0.0f Myr" % (time_values[o][0],time_values[o][-1]), fontsize=16)

ax1.set_ylim(200, 1250)
ax2.set_ylim(1e-5, 5e-1)
ax3.set_ylim(5e3, 5e7)

ax1.set_xlim(0.0, 20)
ax2.set_xlim(0.0, 20)
ax3.set_xlim(0.0, 20)

ax1.legend(loc="lower right", fontsize=16)

ax1.set_ylabel("Radial Velocity [km/s]",  fontsize=16)
ax2.set_ylabel(r"Density [$\rm cm^{-3}$]",  fontsize=16)
ax3.set_ylabel("Temperature [K]", fontsize=16)

ax3.set_xlabel("Radius [kpc]",  fontsize=16)
ax1.set_xticks([])
ax2.set_xticks([])


ax1.tick_params(axis='y', which='major', labelsize=13)
ax2.tick_params(axis='y', which='major', labelsize=13)
ax3.tick_params(axis='both', which='major', labelsize=13)

plt.tight_layout(w_pad=0, h_pad=0)

plt.savefig("./cold_gas_fractions/duty_cycle_variations.pdf", dpi=200, bbox_inches='tight')
plt.show()


### Total cold gas fractions vs time and binned cold gas fraction vs radius, averaged over time

In [ ]:

fig = plt.figure(figsize=(14, 6))
fig.set_rasterized(True)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for o, output in enumerate(outputs):
    ax1.semilogy(np.array(time_values[o]), fraction_cold_gas_time[o], label=labeling[o], linestyle="solid", color=coloring[o])

l1 = ax1.legend(loc="lower right", fontsize=13)
# l2 = ax1.legend(r_labels, r_names, handler_map={tuple: HandlerTuple(ndivide=None)}, loc="upper left", fontsize="medium")
ax1.add_artist(l1)
ax1.tick_params(axis='both', which='major', labelsize="large")
ax1.set(xlim=(0, 50), ylim=(5e-5, 5))
ax1.set_xlabel("Time [Myr]", fontsize=14)
ax1.set_ylabel(r"$\rm M_{cold} /M_{total}$", fontsize=14)
print(time_values)
for o, output in enumerate(outputs):
    ax2.semilogy(dbins[:-1], fraction_cold_gas_r[o], color=coloring[o], label=labeling[o])
    ax2.set(xlim=(0.5, 30), ylim=(5e-5, 1e-1))
ax2.tick_params(axis='both', which='major', labelsize="large")

# ax1.text(1.0, 2, "Time Range: 0 - 50 Myr", fontsize=14, bbox=dict(facecolor='white'))

ax1.text(1.0, 2,"Time Range = %0.0f - %0.0f Myr" % (time_values[o][0],time_values[o][-1]), fontsize="large")

ax2.legend(loc="upper right", fontsize=13)
ax2.set_xlabel("Radius [kpc]", fontsize=14)
ax2.set_ylabel(r"$\rm M_{cold} /M_{total}$", fontsize=14)
plt.savefig("./cold_gas_fraction_time_rad_variation.pdf",bbox_inches='tight', dpi=300)
plt.show()